In [ ]:
# Check NA values
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns])

In [ ]:
stringIndexer = StringIndexer()\
                 .setInputCol ("request_id")\
                 .setOutputCol ("request_id_index")

trans_request = stringIndexer.fit(df_click)
trans_request_df=trans_request.transform(df_click)
trans_request_df.show(10,False)

# VectorAssembler

In [ ]:
assembler = VectorAssembler()\
         .setInputCols (["Age_encoded","Pregnancies","Glucose",
                         "BloodPressure","SkinThickness",\
                         "Insulin","BMI","DiabetesPedigreeFunction"])\
         .setOutputCol ("vectorized_features")
        

assembler_df=assembler.transform(encoder_df)

# StandardScaler

In [ ]:
scaler = StandardScaler()\
         .setInputCol ("vectorized_features")\
         .setOutputCol ("features")
        
scaler_model=scaler.fit(label_indexer_df)
scaler_df=scaler_model.transform(label_indexer_df)
scalar_df.show(10,False)

# Model Training

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=5)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)

# One Hot encoder

In [ ]:
encoder = OneHotEncoderEstimator()\
         .setInputCols (["Age_udfIndex"])\
         .setOutputCols (["Age_encoded"])

encoder_model=encoder.fit(Age_udfIndex_df)
encoder_df=encoder_model.transform(Age_udfIndex_df)